# Bee Walk Video Track

To use in python, we import the `BeeTrack` class from beewalkvideotrack, passing it the mp4 video file we want to process.

In [1]:
from beewalkvideotrack import BeeTrack
import numpy as np

In [ ]:
bt = BeeTrack('demovideo.mp4',fromframe=0,toframe=100)
bt.getpath()
bt.compute_features()
bt.makemovie('demoresult.mp4')

Downsampling by 10
Loading frames
.Using 'averaging' downsampling
.................................................................

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.imshow(bt.first_fromframe)